# Plot Mobility Attack

This notebook captures the essense of our two-phase attack in reconstructing mobile trajectories from transmission metadata. Our attack is carried out on the campus of Peking University in Beijing, China across area that is 800m wide and 950m long. We simulated opportunistic connections between 60 stationary gateways and 12 mobile sensors, and collect metadata on connection times, gateway identity, and sensor identity.

The first phase takes the collected metadata and assumes we have the knowledge of a few of the gateways' locations. In practice, this data is easy to acquire as a network provider simply by deploying a few gateways in opportune areas (ideally with high traffic). We then attempt to reconstruct the locations of all other gateways in the area. We compare our predictions against the ground truth location information to see how well we did.

The second phase takes these predicted locations and reconstructs the mobile trajectories of the 12 sensors based on the sequence of gateways they connect to over time. We compare our predicted trajectory against the ground truth trajectory to see how well we did.

By carrying out this attack, we demonstrate the feasibility of mining rich location data from minimal network metadata and sparse knowledge of gateway locations. As such information reveals personal activity patterns and can be used in subsequent deanonymization attacks, it is imperative to hide this information in subsequent deployments of opportunistic networks.

# The Data

This section loads and visualizes our ground truth gateway locations and sensor trajectories. 

TODO: basically do `visualizing_success` here.

For the `sensor_df`, `norm_secs (secs)` are sensor values stitched together, `time (days)` reflects the difference. But starts at 0. `time (secs)` same as days but in seconds. 

## Phase 1

We start this phase with knowledge of the locations of a few (~3) gateways. Using this knowledge along with the information from the metadata, we construct relative distances between gateways close to each other. We make the simplifying assumption that the sensors move at a constant speed.

Lines that can be edited to provide qualitatively different results will be marked with a `# !!` comment for easier idenfitication.

In [1]:
## Import the libraries.
import glob
import numpy as np
import pandas as pd
from tqdm import tqdm
from matplotlib import pyplot as plt
from scipy.optimize import least_squares

In [ ]:
## Construct a matrix of pairwise distances. This matrix may be sparse.

# This is the list of identifiers for the gateways that we know the positions of.
known_ids = [21, 25, 47, 38, 7, 46, 2, 36, 10, 53, 34, 45, 28, 48] # !!
num_known = len(known_ids)

# Load in the ground truth positions of each stationary gateway.
truth_df = pd.read_csv('../smol_truth/sensors.csv')
num_gateways = len(truth_df)

# Iterate through all traces and generate distance matrices.
dist_diffs_list = []
unscaled_diffs_list = []
agg_speed = 0
for sensor_path in tqdm(glob.iglob('../smol_sim/chopped_mules/*.csv')):
    # Import the simulated mobile sensor connection metadata.
    sensor_df = pd.read_csv(sensor_path)

    # Calculate real-time seconds to avoid aggressive time-skips.
    secs_per_day = 86400
    sensor_df['time (secs)'] = (sensor_df['time (days)'] - sensor_df['time (days)'].iloc[0]) * secs_per_day

    # Subsample the connection data to take samples that are wider apart in time.
    sample_period = 8 # !!
    smol_df = sensor_df.loc[(sensor_df['time (secs)'].round() % sample_period) == 0]

    # Construct lists that hold the times taken to travel between each pair of gateways.
    time_diff_lists = [[[] for j in range(num_gateways)] for i in range(num_gateways)]
    # Do not consider relations between gateways further than cap_time seconds apart.
    cap_time = 2 * 60 # !!
    for i in range(len(smol_df)-1):
        cur_gateway = int(smol_df['sensor ID'].iloc[i])
        cur_time = smol_df['time (secs)'].iloc[i]

        j = 1
        next_gateway = int(smol_df['sensor ID'].iloc[i+j])
        next_time = smol_df['time (secs)'].iloc[i+j]
        time_diff = next_time - cur_time

        while time_diff < cap_time:
            time_diff_lists[cur_gateway][next_gateway].append(time_diff)
            time_diff_lists[next_gateway][cur_gateway].append(time_diff)

            j += 1
            if i+j >= len(smol_df):
                time_diff = cap_time
            else:
                next_gateway = int(smol_df['sensor ID'].iloc[i+j])
                next_time = smol_df['time (secs)'].iloc[i+j]
                time_diff = next_time - cur_time

    # Consolidate the lists of times into a matrix of pairwise times between adjacent gateways.
    # If we do not have any time values between two gateways, we use -1 to indicate no value.
    time_diffs = np.full((num_gateways, num_gateways), -1.0)
    for i in range(num_gateways-1):
        for j in range(i+1, num_gateways):
            times_list = time_diff_lists[i][j]
            if len(times_list) > 0:
                # We expect shorter times to be more representative of direct distance between gateways.
                time_diffs[[i,j],[j,i]] = np.percentile(times_list, 5) # !!

    # We use the time between known gateway positions to get an estimate for the speed of the mobile sensor.
    true_dists = np.zeros((num_known, num_known))
    for i in range(num_known):
        for j in range(num_known):
            dist = np.linalg.norm((truth_df[['x (m)', 'y (m)']].iloc[known_ids[i]] - truth_df[['x (m)', 'y (m)']].iloc[known_ids[j]]))
            true_dists[[i,j],[j,i]] = dist
    # Speed = distance / time. We assume the sensor travels at a constant speed.
    speeds = np.divide(true_dists, time_diffs[known_ids,:][:,known_ids])
    # If there is no time values to calculate speed with, we store this matrix and scale it later.
    if (speeds <= 0).all():
        unscaled_diffs_list.append(time_diffs)
    else:
        ave_speed = np.mean(speeds[speeds > 0]) # !!
        agg_speed += ave_speed
        # Using this estimate of speed, we can then estimate the pairwise distance matrix.
        dist_diffs = time_diffs * ave_speed
        dist_diffs_list.append(dist_diffs)

# Calculate the average speed over all valid trajectories in order to scale trajectories that
# we did not have enough information for. The assumption is that people in the same area move
# at about the same speed, which is completely invalid, but hopefully works well enough in practice.
'''
ave_agg_speed = agg_speed / len(dist_diffs_list)
for time_diffs in unscaled_diffs_list:
    dist_diffs_list.append(time_diffs * ave_agg_speed)
'''

# Consolidate the relative distance matrices.
dist_diffs = np.full((num_gateways, num_gateways), -1.0)
for i in range(num_gateways-1):
    for j in range(i+1, num_gateways):
        val_list = []
        for dist_mat in dist_diffs_list:
            if dist_mat[i,j] > 0:
                val_list.append(dist_mat[i, j])
        if len(val_list) > 0:
            dist_diffs[i,j] = np.median(val_list) # !!

## Our pairwise distance matrix is called `dist_diffs`

230it [01:05,  3.52it/s]

In [ ]:
print(len(unscaled_diffs_list))

In [ ]:
## Construct predictions for the positions of each gateway by making iterative informed guesses.
# Try num_attempts random beginnings and select the results with the minimum loss.
num_attempts = 20
losses = np.zeros((num_attempts, num_gateways))
guess_mat = np.zeros((num_attempts, num_gateways, 2))
for attempt in tqdm(range(num_attempts)):
    # Randomly instantiate our position guesses within the area of interest.
    guesses = np.vstack([np.random.uniform(low=0.0, high=800.0, size=num_gateways),
                         np.random.uniform(low=0.0, high=950.0, size=num_gateways)]).T # !!
    # Instantiate position guesses to be the middle of the area of interest.
    #guesses = np.tile(np.array([[400, 475]]), [num_gateways, 1])

    # Populate our guesses with the positions for our known gateways.
    for known_id in known_ids:
        guesses[known_id] = truth_df[['x (m)','y (m)']].loc[truth_df['sensor ID'] == known_id]
    guess_mat[attempt] = guesses
        
    # Describe an error function to optimize.
    dist_diff = None
    def dist_err(point):
        # Have less confidence near unknown gateway locations.
        unknown_weight = 0.05 # !!
        errs = []
        for i in range(num_gateways):
            # Expected distance based on metadata calculations.
            dist = dist_diff[i]
            if dist > 0:
                # Current distance based on predicted positions.
                point_dist = np.linalg.norm(point - guesses[i])
                if i in known_ids:
                    errs.append(point_dist - dist)
                else:
                    errs.append((point_dist - dist)*unknown_weight)
        return np.array(errs)
    
    def known_err(point):
        # Only give errors from known gateway positions.
        errs = []
        for i in range(num_gateways):
            dist = dist_diff[i]
            if dist > 0 and i in known_ids:
                point_dist = np.linalg.norm(point - guesses[i])
                errs.append(point_dist - dist)
        if errs:
            return np.array(errs)
        return 0.0

    # Guess based only on distances from known positions
    for i in range(num_gateways):
        if i in known_ids:
            continue
        dist_diff = dist_diffs[i]
        guess_mat[attempt, i] = least_squares(known_err, guess_mat[attempt, i],
                                              bounds=(np.array([0.0, 0.0]), np.array([800.0, 950.0])), # !!
                                              loss='soft_l1').x # !!
    
    # Iteratively predict gateway positions to minimize the error function.
    num_iterations = 10 # !!
    for k in range(num_iterations):
        for i in range(num_gateways):
            # Do not need to predict the position of known gateways.
            if i in known_ids:
                continue
            dist_diff = dist_diffs[i]
            result = least_squares(dist_err, guess_mat[attempt, i],
                                   bounds=(np.array([0.0, 0.0]), np.array([800.0, 950.0])), # !!
                                   loss='soft_l1') # !!
            guess_mat[attempt, i] = result.x
            losses[attempt, i] = result.cost

# Consolidate the predicted positions
guess = np.array([guess_mat[opt, ind] for ind, opt in enumerate(np.argmin(losses, axis=0))])
#guess = np.mean(guess_mat, axis=0)

# Do one last optimization.
for i in range(num_gateways):
    if i in known_ids:
        continue
    dist_diff = dist_diffs[i]
    guess[i] = least_squares(known_err, guess[i],
                             bounds=(np.array([0.0, 0.0]), np.array([800.0, 950.0])), # !!
                             loss='soft_l1').x # !!
    
## Our final guesses for gateway positions are in `guess`

In [ ]:
## Analyze our results numerically.

errors = guess - truth_df[['x (m)', 'y (m)']]
dist_errors = np.sort(np.linalg.norm(errors, axis=1))

plt.figure(figsize=(9, 6))
plt.title("Euclidean error of location predictions")
plt.xlabel("gateway IDs (resorted)")
plt.ylabel("error (m)")
plt.plot(dist_errors)
plt.show()

In [ ]:
## Visualize the results.
# Known gateway positions are black.
# Ground truth (unknown) gateway positions are green.
# Predicted (unknown) gateway positions are red.

# Setup the figure.
plt.figure(figsize=(10,12))
plt.title("Guessing Gateway Locations!")
plt.xlim([0, 800])
plt.ylim([0, 950])
ax = plt.gca()
ax.set_aspect("equal")

# Plot ground truth gateway positions.
for index, row in truth_df.iterrows():
    if index not in known_ids:
        ax.add_patch(plt.Circle((row['x (m)'], row['y (m)']), row['radius (m)'], alpha = 0.4, color='g'))
        plt.text(row['x (m)'], row['y (m)'], int(row['sensor ID']))

# Plot known and predicted gateway positions.
for index, pos in enumerate(guess):
    if index in known_ids:
        ax.add_patch(plt.Circle(pos, 15, alpha=0.6, color='k'))
        plt.text(pos[0], pos[1], index)
    else:
        ax.add_patch(plt.Circle(pos, 15, alpha=0.4, color='r'))
        plt.text(pos[0], pos[1], "{}?".format(index))

# Display the figure.
plt.show()

## Phase 2

After predicting the gateway locations, we then do a mobile trajectory reconstruction attack.

In [ ]:
# Import libraries.
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.interpolate import interp1d
from scipy.ndimage import gaussian_filter1d

In [ ]:
def path_prediction(mobile_id='167', sample_period=4, spline_type='linear', filter_std=20):
    # Read in path simulation and ground truth files
    sim = pd.read_csv("../smol_sim/mules/{}.csv".format(mobile_id))
    truth = pd.read_csv("../smol_truth/mules/{}.csv".format(mobile_id))
    
    # Downsample the simulation.
    down_sim = sim.loc[(sim['norm_secs (secs)'].round() % sample_period) == 0]
    
    # Compile positions of seen gateways.
    t = []
    sim_x = []
    sim_y = []

    for index, row in down_sim.iterrows():
        t.append(row['norm_secs (secs)'])
        sim_x.append(guess[int(row['sensor ID']), 0])
        sim_y.append(guess[int(row['sensor ID']), 1])
    
    
    # The goal is to spline. However, splines dont work well with multiple values
    # at one point, which happens when we see multiple gateways at the same time.
    # Thus, if we see multiple gateways at the same time, we average the positions.
    ct = []
    cx = []
    cy = []

    counter = 0
    while counter < len(t):
        cur_t = t[counter]
        cum_x = [sim_x[counter]]
        cum_y = [sim_y[counter]]
        counter += 1
        while counter < len(t) and t[counter] == cur_t:
            cum_x.append(sim_x[counter])
            cum_y.append(sim_y[counter])
            counter += 1
        ct.append(cur_t)
        cx.append(sum(cum_x) / len(cum_x))
        cy.append(sum(cum_y) / len(cum_y))
    
    
    # Calculate our predictions.
    fx = interp1d(ct, cx, kind=spline_type)
    fy = interp1d(ct, cy, kind=spline_type)
    tt = truth.loc[(truth['norm_secs (secs)'] > ct[0]) & 
                   (truth['norm_secs (secs)'] < ct[-1])]['norm_secs (secs)'].to_numpy()
    
    # Predict points based on a spline and apply a Gaussian filter to smooth the points.
    px = gaussian_filter1d(fx(tt), filter_std)
    py = gaussian_filter1d(fy(tt), filter_std)

    # Calculate the Euclidean error of the prediction for each truth point.
    err = []
    for i in range(len(tt)):
        ex = px[i] - truth.loc[truth['norm_secs (secs)'] == tt[i]].iloc[0]['x (m)']
        ey = py[i] - truth.loc[truth['norm_secs (secs)'] == tt[i]].iloc[0]['y (m)']
        err.append(np.linalg.norm([ex, ey]))

    
    # Plot the results.
    # Set some parameters.
    truth_marker = 'k,:'
    truth_alpha = 0.4
    prediction_color = 'g'
    prediction_marker = 'g,:'
    prediction_alpha = 0.4
    error_marker = 'b,-'
    gateway_marker = 'rx'
    gateway_size = 10
    
    # Plot x over time.
    plt.figure(figsize=(15, 7))
    plt.title('X position of {} over time'.format(mobile_id))
    plt.xlabel('time (secs)')
    plt.ylabel('x (m)')
    plt.plot(truth['norm_secs (secs)'], truth['x (m)'], truth_marker, alpha=truth_alpha, label='truth')
    plt.plot(tt, px, prediction_marker, alpha=prediction_alpha, label='{} spline prediction'.format(spline_type))
    plt.plot(t, sim_x, gateway_marker, markersize=gateway_size, label='seen gateways')
    for index, row in truth_df.iterrows():
        if index not in known_ids:
            plt.Circle((row['x (m)'], row['y (m)']), row['radius (m)'], alpha = 0.4, color='g')
            
    plt.legend()
    plt.show()
    
    # Plot y over time.
    plt.figure(figsize=(15, 7))
    plt.title('Y position of {} over time'.format(mobile_id))
    plt.xlabel('time (secs)')
    plt.ylabel('y (m)')
    plt.plot(truth['norm_secs (secs)'], truth['y (m)'], truth_marker, alpha=truth_alpha, label='truth')
    plt.plot(tt, py, prediction_marker, alpha=prediction_alpha, label='{} spline prediction'.format(spline_type))
    plt.plot(t, sim_y, gateway_marker, markersize=gateway_size, label='seen gateways')
    plt.legend()
    plt.show()
    
    # Plot the error over time.
    plt.figure(figsize=(15, 7))
    plt.title("Euclidean error of a {} spline approximation for {}".format(spline_type, mobile_id))
    plt.xlabel('time (sec)')
    plt.ylabel('error (m)')
    plt.plot(tt, err, error_marker, label="Euclidean error")
    plt.vlines(ct, 0, 900, colors=prediction_color, alpha=prediction_alpha, label="transmission times")
    plt.legend()
    plt.plot()
    
    # Plot the trajectories v.s. the prediction
    plt.figure(figsize=(15, 7))
    plt.gca().set_aspect("equal")
    plt.title("Prediction v.s. Ground Truth for {}".format(mobile_id))
    plt.xlabel("x (m)")
    plt.ylabel("y (m)")
    plt.plot(truth['x (m)'], truth['y (m)'], truth_marker, label="ground truth trajectory", alpha=truth_alpha)
    plt.plot(px, py, prediction_marker, alpha=prediction_alpha, label="predicted trajectory")
    plt.plot(sim_x, sim_y, gateway_marker, markersize=gateway_size, label='seen gateways')
    plt.legend()
    plt.show()

## Results

Man, these results make me so sad. 

In [ ]:
path_prediction("011")